# Part 1 of the Course of NN

## 1) Importing the packages and data pre-processing

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

See the data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [11]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 2) Adjust format and keep important information

In [4]:
df.drop('RowNumber', axis = 1, inplace= True)
df.drop('CustomerId', axis = 1, inplace= True)
df.drop('Surname', axis = 1, inplace= True)

genders = {'Male': 1, 'Female': 0}
df['Gender'] = df['Gender'].map(genders)

In [5]:
#Since there are only 3 possible countries, Spain, France, and Germany, we can do 2 things. Use a dictionary where Spain is closer to France than
#to Germany ... or just user OneHotEncoder and avoid assumptions. In this case I will go with the 2nd one.
geo_dummies = pd.get_dummies(df['Geography'], prefix='Geography')

df = pd.concat([df, geo_dummies], axis = 1)

df.drop('Geography', axis = 1, inplace = True)

In [15]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,0,1


## 3) Fit the data and variables that the model is going to use

In [10]:
target = df['Exited']
features = df.drop('Exited', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.8, random_state=42)

scaler = StandardScaler() #We need to use a scaler in order to give this input to the NN.

X_train_scaled = scaler.fit_transform(X_train) #First we fit the scaling that is going to be made
X_test_scaled = scaler.transform(X_test) #Then we apply the transformation to the test set

## IMPORTANT. In order to avoid data leaking we need to fit with training set and then transform to the test one. Doing something like: 
# features_scaled = scaler.fit_transform(features) is wrong and it is going to lead to a data leaking problem.

## 4) Build the Neural Network

In [12]:
import tensorflow as tf

nn = tf.keras.models.Sequential() # Create the neural network, in this case with the Sequential model which  
#allows you to build neural network models layer by layer. It is useful when you have a linear architecture, that is, each layer has exactly one input and one output.

nn.add(tf.keras.layers.Dense(units= 12, activation='relu'))  # Add first hidden layer. The number of neurons may vary when building a NN. However, I personally, 
#like to start with the same amount of columns of "features"

nn.add(tf.keras.layers.Dense(units= 12, activation='relu')) # Add second hidden layer. We use the Rectified Linear Unit(relu) function. 
#RelU does not suffer from gradient fading problems, which makes it easier to train deep networks.

nn.add(tf.keras.layers.Dense(units= 1, activation='sigmoid')) # Add the output hidden layer. Since we he want to predict only one binary variable, the best option is 1. 
#if we would like to get something different, we would have to add more neurons. The activation function has to change, in this case, if we use the sigmoid one,
#it is going to give us the probabilitly of the variable taking the value of 1.

## 5) Train the Neural Network